# Dash Caching

In [4]:
import os, sys; from importlib import reload;
from utils.constants import *
%cd {os.environ['PROJECT_PATH']}

import plotly.express as px
import jupyter_dash

/app


- Load Data 
    - Input -> ButtonClick
    - Saves dataframe to Cache
- Plotting Functions
    - Input -> dataframe changes & fields
    - Fetch from cache
    - Output to Chart
- Modeling
    - Input -> ButtonClick
    - Fetch Data
    - Train Model
    - Save model to cache
- 

In [5]:
### Imports
import json, sys
from dash import dcc, html
from dash.dependencies import Input, Output, State

from src.dash_app.apps import dash_global

df = px.data.iris()

### App Setup
app = jupyter_dash.JupyterDash(__name__)

app.layout = html.Div([
    html.H1('Dash & DataCaching', style={"textAlign": "center"}),
    html.H3('Load Data', style={"textAlign": "center"}),
    html.Button(id='button-load', children=['Load Data'], n_clicks=0),
    dcc.Store('raw-data'),
    
    html.H3('Visualize', style={"textAlign": "center"}),
    dcc.Dropdown(id='species_ids', multi=True,
                 options=[{'value':i,'label':i} for i in df.species_id.unique()]),
    html.Div(id='viz'),
    dcc.Store('split-data'),
    
    html.H3('Modeling', style={"textAlign": "center"}),
    html.Button(id='button-train', children=['Train Model'], n_clicks=0),
    dcc.Store('model-binary'),
    
    html.Div(id='logs-div'),
])


### Cache Functions
CACHE = {}
def set_cache(obj):
    global CACHE
    try:
        hash_key = hash(obj)
    except:
        hash_key = len(obj)
    CACHE[hash_key] = obj
    return hash_key

def get_cache(hash_key):
    global CACHE
    return CACHE[hash_key]

def log(*msg): print(*msg,file=sys.stderr)


### Load Data
@app.callback(
    Output('raw-data','data'),
    Input('button-load','n_clicks')
)
def button_clicked(n):
    log('button_clicked', n)
    return set_cache(df.copy())


### Viz Functions
@app.callback(
    Output('viz','children'),
    Input('raw-data','data'), Input('species_ids','value')
)
def trigger_viz(data, species_ids):
    log('trigger_viz', data, species_ids)
    # if not data : pass;
    dff2 = get_cache(data)
    return dcc.Graph(
        figure=px.scatter_3d(dff2, 'sepal_length','petal_length','sepal_width', color='species')
    )



### Modeling & Save
import numpy as np

@app.callback(
    Output('model-binary','data'),
    Input('button-train','n_clicks'), State('raw-data','data')
)
def trigger_train_model(n, data):
    log('trigger_train_model',n, data)
    if (n < 1) or data == None : 
        print('trigger_train_model skipping..',file=sys.stderr)
        return None
    dff = get_cache(data)
    
    model = lambda x : np.rand.rand()
    return set_cache(model)


### Run App
if __name__ == '__main__':
    app.run_server(mode='inline',
                    port=8501, 
                    host="0.0.0.0",
                    debug=True, #**{'width': '1200px', 'height': '800px'}
      # assets_external_path="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css"
    )

/usr/local/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



button_clicked 0
trigger_train_model 0 None
trigger_train_model skipping..
trigger_viz 150 None
button_clicked 1
trigger_viz 150 None
trigger_viz 150 [2]
trigger_viz 150 [2, 3]
